# VGG Pretrain Balance

Accu 87%


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torchvision import models

# Define the transforms
trans = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
])

# Load the dataset
cifar_train = datasets.CIFAR10('cifar', train=True, transform=trans,download=True)
cifar_test = datasets.CIFAR10('cifar', train=False, transform=trans,download=True)


Files already downloaded and verified
Files already downloaded and verified


In [3]:

# Define the data loaders
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=64, shuffle=False)

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = models.vgg19(pretrained=True)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    # print(f"epoch: {epoch}")
    for i, data in enumerate(trainloader, 0):
        # print(f"i: {i}")
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

cuda


/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Accuracy of the network on the 10000 test images: 86 %


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the test data
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=64, shuffle=False)

# Evaluate the model
y_true = []
y_pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true += labels.tolist()
        y_pred += predicted.tolist()

# Compute the metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print('Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}'.format(acc, prec, rec, f1))

Accuracy: 0.8671, Precision: 0.8700, Recall: 0.8671, F1: 0.8673
